This is the IOHMM model with the parameters learned in a semi-supervised way. By using some labeled data, we force the learning process in a certain direction. The unlabeled data will be estimated using EM algorithm iteratively. See notes in http://pages.cs.wisc.edu/~jerryzhu/pub/sslicml07.pdf

# Example use of SemiSupervised_IOHMM 

In [1]:
from __future__ import  division
import numpy as np
from copy import deepcopy
import sys
sys.path.append('./main')
sys.path.append('./auxiliary')
from IOHMM import SemiSupervisedIOHMM, SemiSupervisedIOHMMMapReduce
from SupervisedModels import LM, MNLP, MNLD
from scipy.misc import logsumexp
import pandas as pd
import warnings
warnings.simplefilter("ignore")

## Speed data

### The sequence data

In [2]:
speed = pd.read_csv('data/speed.csv')
speed.head()

,Unnamed: 0,rt,corr,Pacc,prev
0,1,6.456770,cor,0.0,inc
1,2,5.602119,cor,0.0,cor
2,3,6.253829,inc,0.0,cor
3,4,5.451038,inc,0.0,inc
4,5,5.872118,inc,0.0,inc


### The labeled states

In our structure of the code, the states should be a dictionary, the key is the index in the sequence (e.g. 0, 5) and the value is a one-out-of-n code of array where the kth value is 1 if the hidden state is k. n is the number of states in total.

In the following example, we assume that the "corr" column gives the correct hidden states. Here we assume only the first half of the sequence is labeled.

In [3]:
speed.shape[0]

439

In [4]:
states = {}
corr = np.array(speed['corr'])
for i in range(int(len(corr)/2)):
    state = np.zeros((4,))
    if corr[i] == 'cor':
        states[i] = np.array([0,1,0,0])
        speed.set_value(i, 'rt', 1)
    else:
        states[i] = np.array([1,0,0,0])
        speed.set_value(i, 'rt', 0)

In [5]:
speed.shape[0]

439

In [6]:
len(states)

219

## Setting up the model

In [7]:
SHMM = SemiSupervisedIOHMM(num_states=4)
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])
SHMM.setData([[speed, states]])

## Start training

In [8]:
SHMM.train()

-854.245242987
-553.133617705
-8.29386187162
21417.0783514
-160.487640865
-159.811838082
-157.948337864
-151.860179181
-136.414814375
-89.0762221538
-54.2618483495
-46.0672472774
-44.8127554225
-44.4773996276
-44.416526008
-44.4040927806
-44.4014929881
-44.4009453097
-44.4008297398
-44.4008053438


## See coefficients

In [9]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.42176108  0.5332247   0.02250711  0.02250711]]
[[ 0.15113613  0.81203853  0.01918793  0.01763742]]


In [10]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[2][0].coef
print SHMM.model_emissions[3][0].coef

[-0.]
[ 1.]
[ 5.47279282]
[ 6.3909629]


In [11]:
print np.sqrt(SHMM.model_emissions[0][0].dispersion)
print np.sqrt(SHMM.model_emissions[1][0].dispersion)
print np.sqrt(SHMM.model_emissions[2][0].dispersion)
print np.sqrt(SHMM.model_emissions[3][0].dispersion)

0.0
0.0
0.182277327167
0.2246206096


## MapReduce Version

In [12]:
sc.stop()
sc = SparkContext(appName="Python_UnSupervised_IOHMM_MapReduce", pyFiles=[
    './auxiliary/HMM.py',
    './auxiliary/SupervisedModels.py',
    './auxiliary/family.py',
    './main/IOHMM.py'])

In [13]:
speed = pd.read_csv('data/speed.csv')
states = {}
corr = np.array(speed['corr'])
for i in range(int(len(corr)/2)):
    state = np.zeros((4,))
    if corr[i] == 'cor':
        states[i] = np.array([0,1,0,0])
        speed.set_value(i, 'rt', 1)
    else:
        states[i] = np.array([1,0,0,0])
        speed.set_value(i, 'rt', 0)

In [14]:
indexes = [(1,1), (2,1)]
RDD = sc.parallelize(indexes)
dfs_states = RDD.mapValues(lambda v: [speed, states])

In [15]:
SHMM = SemiSupervisedIOHMMMapReduce(num_states=4, max_EM_iter=100, EM_tol=1e-6)
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])
SHMM.setData(dfs_states)
SHMM.train()
print 'done'

-1190.24612273
2282.46051374
-510.222272458
-182.177298438
-299.190985502
-272.632075225
-177.862077443
-98.6115486712
-87.9166057873
-91.4199278253
-90.9865899289
-90.8911350999
-90.8663075776
-90.859583132
-90.8577407855
-90.8572344141
-90.8570951151
-90.8570567858
-90.8570462384
-90.857043336
-90.8570425372
done


In [16]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[2][0].coef
print SHMM.model_emissions[3][0].coef

[-0.]
[ 1.]
[ 5.47205369]
[ 6.39057687]


In [17]:
sc.stop()